# Getting Started with AWS Bedrock Agent Library

This notebook demonstrates how to use the bedrock_agent library from the src/utils directory to create and interact with AWS Bedrock Agents. This library provides high-level Pythonic objects such as Agent, Guardrail, and Tool to simplify building with Bedrock Agents in Python.

## Importing the library

In [ ]:
import os
import sys

sys.path.append("../..")
sys.path.append("../../src")
from utils.bedrock_agent import Agent, Tool, ParameterSchema, ParamType, Task

## Basic Agent operations - create, test existence, invoke, delete

Let's start by creating a simple 'Hello World' agent. This agent will demonstrate the basic components needed to create and interact with a Bedrock Agent.

### Simple create and invoke

In [ ]:
Agent.set_force_recreate_default(True)
agent = Agent.create(
    "poet-agent", "Write poems", "Write poems as directed by the user", "write poems"
)
print(agent.invoke("Please write a poem about ice hockey"))

### Check for the agent's existence
Note that the agent is persisted in Bedrock:

In [ ]:
print(Agent.exists("poet-agent"))

### Delete the agent

In [ ]:
Agent.delete_by_name("poet-agent")

## More advanced Agent creation
You can specify more information when creating an Agent, and can create them in other ways, such as from a YAML configuration file. Alteratively, you can also modify the agent (ex. attach a Tool) after its creation.

In [ ]:
calculator_agent = Agent.create(
    name="calculator_agent",
    role="Perform calculations",
    goal="Calculate responses to provided questions",
    instructions="""If the question involves a numerical calculation, execute it using the 
                                     code interpreter as needed. If it is not a calculation, reply with the
                                     phrase 'I can help you with calculations. Do you have a calcalation for me?
                                     """,
    llm="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    code_interpreter=True,  # lets us write code to do accurate calculations
    verbose=True,
)
print(calculator_agent.invoke("What is sin(.268) * pi/2?"))

### Creating an Agent from a YAML template
It is often convenient, particularly when working with multiple agents, to define the agents in a YAML file. The file can contain multiple agent definitions and is, by default, 'agents.yaml' (although you can specify a different filename):

In [ ]:
Agent.set_force_recreate_default(True)
test_agent = Agent.create_from_yaml("test_agent")

Tha YAML file contains definitions: 

## Working with Tools

Agents can be enhanced with tools that allow them to perform specific actions. Tools can be attached at Agent create time or later. They can be created from local code files, or created directly from Python functions.

### Creating tools from code
You can create a Tool from a Python function that is annotated with type hints. Note this currently materializes a lambda created from the function:

In [ ]:
def mask_string(input_string: str) -> str:
    """Masks a string by replacing all but the last four characters with asterisks."""
    if len(input_string) <= 4:
        return input_string
    else:
        return "*" * (len(input_string) - 4) + input_string[-4:]

In [ ]:
test_agent.attach_tool_from_function(mask_string)
test_agent.prepare()
test_agent.invoke("Please mask this string: 077-29-2932")

### Creating a Tool via a ParameterSchema
If you have a lambda that is not in Python and/or does not have type hints, you can create a ParameterSchema for it to define the parameters needed. From that and a code file, you can create a Tool, and then attach it to your Agent.

In [ ]:
schema = ParameterSchema.create_with_values(
    name="input_string",
    parameter_type=ParamType.STRING,
    description="String to transmorgify",
    required=False,
)
transmorgifier_tool = Tool.create(
    "transmorgifier",
    schema=schema,
    code_file="lambda_transmorgify_string.py",
    description="transmorgifies a string",
)
test_agent.attach_tool(transmorgifier_tool)
print(
    test_agent.invoke(
        "Please transmorgify this string: Who can say what the rain may bring?"
    )
)

### Defining Tools at Agent creation
You can define the Tool with the agent, if its parameter schema is available:

In [ ]:
test_user_data_agent = Agent.create(
    name="test_user_data_agent",
    role="Accesser of user data",
    goal="Find user profile data and return it",
    instructions="Use the provided tool to find user profile data and return it",
    tool_code="lambda_lookup_user_profile.py",
    tool_defs=[
        {
            "name": "lookup_user_profile",
            "description": "Return user profile data as a JSON structure. Requires an input user id",
            "parameters": {
                "user_id": {
                    "description": "The unique identifier of the user",
                    "type": "string",
                    "required": False,
                }
            },
        }
    ],
    llm="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    code_interpreter=False,
    verbose=True,
)

In [ ]:
Agent.delete_by_name("test_user_data_agent")

## Working with Knowledge Bases

Agents can be enhanced with knowledge bases to provide additional context and information.

In [ ]:
# Create a knowledge base (to show attachment to an Agent on creation)
import time
from src.utils.knowledge_base_helper import KnowledgeBasesForAmazonBedrock
import random

# Create a quick KB for demonstration purposes
kb_helper = KnowledgeBasesForAmazonBedrock()
kb_name = "mortgage-test-kb"
random_string = "".join(random.choices("0123456789", k=4))
bucket_name = f"test-kb-bucket-{random_string}"
kb_id, ds_id = kb_helper.create_or_retrieve_knowledge_base(
    kb_name,
    kb_description="Useful for answering questions about mortgage refinancing and for questions comparing various mortgage types",
    data_bucket_name=bucket_name,
)
# Normally you would upload data to the bucket here

time.sleep(30)  # ensure that the kb is available
kb_helper.synchronize_data(kb_id, ds_id)  # sync knowledge base

### Attach a knowledge base on agent creation

In [ ]:
# Create an agent, specifying the LLM and attaching a KnowledgeBase
general_mortgage_questions = Agent.create(
    name="mortgage_test_agent",
    role="General Mortgage Questions",
    goal="Handle conversations about general mortgage questions, like high level concepts of refinancing or tradeoffs of 15-year vs 30-year terms.",
    instructions="""You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types.""",
    kb_id=kb_id,
    kb_descr="""Use this knowledge base to answer general questions about mortgages, like how to refinance, or the difference between 15-year and 30-year mortgages.""",
    llm="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
)

### Clean up the knowledge base

In [ ]:
kb_helper.delete_kb("mortgage-test-kb")

## Cleanup
Run this cleanup code to delete the agents created above:

In [ ]:
Agent.delete_by_name("mortgage_test_agent")
Agent.delete_by_name("test_agent")
Agent.delete_by_name("calculator_agent")

Delete the lambdas (deleting the Agent and Tools leaves the lambdas in place)

In [ ]:
import boto3
lambda_client = boto3.client('lambda')
lambda_client.delete_function(FunctionName='mask_string')
lambda_client.delete_function(FunctionName='transmorgifier')